In [1]:
import pandas as pd
import json
from environs import Env
from neo4j import GraphDatabase


In [2]:
env = Env()
env.read_env()

EPIGRAPHDB_SERVER = env.str("EPIGRAPHDB_SERVER")
EPIGRAPHDB_USER = "neo4j"
EPIGRAPHDB_PORT = env.str("EPIGRAPHDB_PORT")
EPIGRAPHDB_PASSWORD = env.str("EPIGRAPHDB_PASSWORD")
print(EPIGRAPHDB_PORT)

epigraphdb_driver = GraphDatabase.driver(
    "bolt://{server_name}:{port}".format(
        server_name=EPIGRAPHDB_SERVER, port=EPIGRAPHDB_PORT),
    auth=(EPIGRAPHDB_USER, EPIGRAPHDB_PASSWORD))

session = epigraphdb_driver.session()

37687


In [3]:
def query_to_df(query):
    print(query)
    data=session.run(query).data()
    df = pd.json_normalize(data)
    return(df)

# Looking at literature connections

In [45]:
# get all types of predicate
query="""
MATCH (gwas:Gwas)-[gs:GWAS_TO_LITERATURE_TRIPLE]->(triple:LiteratureTriple) -[sl:SEMMEDDB_TO_LIT]->(lit:Literature) 
return distinct triple.predicate limit 43
"""
df = query_to_df(query)
df



MATCH (gwas:Gwas)-[gs:GWAS_TO_LITERATURE_TRIPLE]->(triple:LiteratureTriple) -[sl:SEMMEDDB_TO_LIT]->(lit:Literature) 
return distinct triple.predicate limit 43



KeyboardInterrupt: 

### Example query "GWAS - literature - GWAS"

In [4]:
# Childhood obesity -> breast cancer

query="""
MATCH (gwas:Gwas)-[gwas_to_lit1:GWAS_TO_LITERATURE_TRIPLE]->(s1:LiteratureTriple) 
-[:SEMMEDDB_OBJ]->(lit_term:LiteratureTerm)<-[:SEMMEDDB_SUB]- (s2:LiteratureTriple)<-[gwas_to_lit2:GWAS_TO_LITERATURE_TRIPLE]-(assoc_gwas:Gwas) 
WHERE gwas.id = "ieu-a-1096" AND assoc_gwas.id = "ieu-a-1126" 
AND gwas_to_lit1.pval < 1e-05 AND gwas_to_lit2.pval < 1e-05 
AND NOT lit_term.type IN ['nusq','dsyn'] 
RETURN distinct gwas.id, gwas.trait, gwas_to_lit1.pval,  s1.predicate, lit_term.name, s2.predicate, gwas_to_lit2.pval, assoc_gwas.id, assoc_gwas.trait 
order by gwas_to_lit2.pval
SKIP 0 LIMIT 1000
"""
df_obesity_lit = query_to_df(query)
df_obesity_lit



MATCH (gwas:Gwas)-[gwas_to_lit1:GWAS_TO_LITERATURE_TRIPLE]->(s1:LiteratureTriple) 
-[:SEMMEDDB_OBJ]->(lit_term:LiteratureTerm)<-[:SEMMEDDB_SUB]- (s2:LiteratureTriple)<-[gwas_to_lit2:GWAS_TO_LITERATURE_TRIPLE]-(assoc_gwas:Gwas) 
WHERE gwas.id = "ieu-a-1096" AND assoc_gwas.id = "ieu-a-1126" 
AND gwas_to_lit1.pval < 1e-05 AND gwas_to_lit2.pval < 1e-05 
AND NOT lit_term.type IN ['nusq','dsyn'] 
RETURN distinct gwas.id, gwas.trait, gwas_to_lit1.pval,  s1.predicate, lit_term.name, s2.predicate, gwas_to_lit2.pval, assoc_gwas.id, assoc_gwas.trait 
order by gwas_to_lit2.pval
SKIP 0 LIMIT 1000



,gwas.id,gwas.trait,gwas_to_lit1.pval,s1.predicate,lit_term.name,s2.predicate,gwas_to_lit2.pval,assoc_gwas.id,assoc_gwas.trait
0,ieu-a-1096,Childhood obesity,8.204412e-06,INHIBITS,H3F3AP6,STIMULATES,2.287763e-14,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
1,ieu-a-1096,Childhood obesity,8.204412e-06,INHIBITS,NSG1,STIMULATES,2.287763e-14,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
2,ieu-a-1096,Childhood obesity,8.204412e-06,INHIBITS,TCEAL1,STIMULATES,2.287763e-14,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
3,ieu-a-1096,Childhood obesity,8.204412e-06,INHIBITS,CDKN1A,STIMULATES,2.287763e-14,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
4,ieu-a-1096,Childhood obesity,8.204412e-06,ASSOCIATED_WITH,Obesity,PREDISPOSES,1.804619e-12,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
...,...,...,...,...,...,...,...,...,...
506,ieu-a-1096,Childhood obesity,4.543086e-07,COEXISTS_WITH,leptin,STIMULATES,5.440435e-06,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
507,ieu-a-1096,Childhood obesity,8.847083e-13,COEXISTS_WITH,Insulin,STIMULATES,5.440435e-06,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
508,ieu-a-1096,Childhood obesity,8.204412e-06,COEXISTS_WITH,testosterone,INTERACTS_WITH,9.004132e-06,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...
509,ieu-a-1096,Childhood obesity,8.204412e-06,COEXISTS_WITH,testosterone,INTERACTS_WITH,9.548222e-06,ieu-a-1126,Breast cancer (Combined Oncoarray; iCOGS; GWAS...


In [ ]:

#ukb-b-4650	Comparative body size at age 10 -> breast cancer


query="""
MATCH (gwas:Gwas)-[gwas_to_lit1:GWAS_TO_LITERATURE_TRIPLE]->(s1:LiteratureTriple) 
-[:SEMMEDDB_OBJ]->(lit_term:LiteratureTerm)<-[:SEMMEDDB_SUB]- (s2:LiteratureTriple)<-[gwas_to_lit2:GWAS_TO_LITERATURE_TRIPLE]-(assoc_gwas:Gwas) 
WHERE gwas.id = "ukb-b-4650" AND assoc_gwas.id = "ieu-a-1126" 
AND gwas_to_lit1.pval < 1e-05 AND gwas_to_lit2.pval < 1e-05 
AND NOT lit_term.type IN ['nusq','dsyn'] 
RETURN distinct gwas.id, gwas.trait, gwas_to_lit1.pval,  s1.predicate, lit_term.name, s2.predicate, gwas_to_lit2.pval, assoc_gwas.id, assoc_gwas.trait 
order by gwas_to_lit2.pval
SKIP 0 LIMIT 1000
"""
df_bodysize_lit = query_to_df(query)
df_bodysize_lit

In [8]:
df_bodysize_lit.reset_index()['lit_term.name'].value_counts()



Obesity                 54
leptin                  10
LEP                      8
metformin                6
ethanol                  2
Hypertensive disease     2
Name: lit_term.name, dtype: int64

In [10]:
df_bodysize_lit['test'] = df_bodysize_lit['s1.predicate'] + " - "+ df_bodysize_lit['lit_term.name'] + " - "+ df_bodysize_lit['s2.predicate']
df_bodysize_lit.reset_index()['test'].value_counts()

TREATS - Obesity - PRODUCES                         10
TREATS - Obesity - PREDISPOSES                      10
INTERACTS_WITH - leptin - STIMULATES                 8
ASSOCIATED_WITH - Obesity - PREDISPOSES              6
INTERACTS_WITH - LEP - STIMULATES                    6
ASSOCIATED_WITH - Obesity - PRODUCES                 6
COEXISTS_WITH - Obesity - PREDISPOSES                6
COEXISTS_WITH - Obesity - PRODUCES                   6
higher_than - metformin - INHIBITS                   3
NEG_ASSOCIATED_WITH - Obesity - PRODUCES             3
NEG_ASSOCIATED_WITH - Obesity - PREDISPOSES          3
INTERACTS_WITH - leptin - INTERACTS_WITH             2
INTERACTS_WITH - LEP - INTERACTS_WITH                2
AUGMENTS - Obesity - PREDISPOSES                     1
PREDISPOSES - Hypertensive disease - PREDISPOSES     1
higher_than - metformin - ASSOCIATED_WITH            1
COEXISTS_WITH - ethanol - INTERACTS_WITH             1
AFFECTS - Obesity - PREDISPOSES                      1
higher_tha

In [11]:
df_obesity_lit['test'] = df_obesity_lit['s1.predicate'] + " - "+ df_obesity_lit['lit_term.name'] + " - "+ df_obesity_lit['s2.predicate']
df_obesity_lit.reset_index()['test'].value_counts()

ASSOCIATED_WITH - Obesity - PREDISPOSES                 71
ASSOCIATED_WITH - Obesity - PRODUCES                    71
TREATS - Obesity - PREDISPOSES                          45
TREATS - Obesity - PRODUCES                             45
COEXISTS_WITH - Obesity - PREDISPOSES                   36
                                                        ..
INHIBITS - H3F3AP6 - COEXISTS_WITH                       1
COEXISTS_WITH - INSR - INTERACTS_WITH                    1
ASSOCIATED_WITH - Hypertensive disease - PREDISPOSES     1
NEG_COEXISTS_WITH - TRANCE protein - INTERACTS_WITH      1
CAUSES - Hypertensive disease - PREDISPOSES              1
Name: test, Length: 70, dtype: int64

In [12]:
df_obesity_lit.reset_index()['lit_term.name'].value_counts()


Obesity                 430
leptin                   20
Hypertensive disease     12
LEP                       8
CDKN1A                    6
Insulin                   6
NSG1                      6
TCEAL1                    6
H3F3AP6                   6
TRANCE protein            2
testosterone              2
folate                    2
HDAC9                     1
INS                       1
TGFB1                     1
TGFB1 protein, human      1
INSR                      1
Name: lit_term.name, dtype: int64

## Testing literature triples


In [9]:
genes_test_df = pd.read_csv("query_results/bc_gwas_expressed_in_breastTissue.tsv", sep = '\t')
genes_test = list(set(genes_test_df['gene.name']))
len(genes_test)

98

In [10]:

GENE_LIST = genes_test # genes expressed in breast tissues with effect in GWAS

# tophits in GWAS -> genes -> expressed in Breast tissue -> map to lit -> triple to other lit term
query = """
        MATCH (g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt:LiteratureTriple)-[]-(l2:LiteratureTerm)
        WHERE g.name in {gene_list}
        WITH  g,lt,l1,l2
        MATCH (lt)-[]-(lit:Literature) 
        RETURN l1.name as gene_as_litterm, lt.predicate as pred, l2.name as obj_as_litterm, lit.id ;
    """.format(gene_list=GENE_LIST)
df= query_to_df(query)
df


        MATCH (g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt:LiteratureTriple)-[]-(l2:LiteratureTerm)
        WHERE g.name in ['NEK10', 'ANKLE1', 'ABO', 'ATG10', 'ADCY3', 'CREB5', 'CHEK2', 'ZNF184', 'HSCB', 'HSPA4', 'KANSL1', 'ELL', 'TET2', 'CBX8', 'MCM8', 'SETBP1', 'SSBP4', 'MKL1', 'PHLDA3', 'EBF1', 'TLR1', 'MLLT10', 'TTC28', 'LGR6', 'PEX14', 'FTO', 'SUB1', 'KCNN4', 'RIN3', 'OR2A1-AS1', 'SF3B4', 'N4BP2L1', 'TRIM46', 'CCDC88C', 'ADCY9', 'LINC-PINT', 'SLC25A22', 'USHBP1', 'ADSSL1', 'TCF7L2', 'VGLL3', 'STXBP4', 'FAM46A', 'MTMR11', 'CDYL2', 'MDM4', 'ASTN2', 'MYEOV', 'ATXN7', 'RANBP9', 'ESR1', 'GRHL1', 'L3MBTL3', 'CDCA7L', 'PLA2G6', 'ULK3', 'ARRDC3', 'AMFR', 'FGFR2', 'CUX1', 'OTUD7B', 'BABAM1', 'KRTCAP2', 'FAM175A', 'NREP', 'CASP8', 'PIDD', 'ZMIZ1', 'SCAMP2', 'GIPR', 'ARHGEF5', 'ZBTB38', 'PIK3R3', 'PDE4D', 'CMSS1', 'RPL23AP53', 'MAGI3', 'LSP1', 'ATAD5', 'ZFPM2', 'SV2A', 'DNAJC1', 'RCCD1', 'TGFBR2', 'PRC1', 'NRIP1', 'TACC2', 'ITPR1', 'GAREM', 'LMX1B', 'RNF115', 'FOXP1', 'FILIP1L', 'V

,gene_as_litterm,pred,obj_as_litterm,lit.id
0,NRIP1,AFFECTS,Obesity,20205832
1,NRIP1,AFFECTS,Obesity,20680503
2,NRIP1,INHIBITS,HOXA5,19862326
3,NRIP1,COEXISTS_WITH,Tretinoin,19862326
4,NRIP1,STIMULATES,"Receptors, Nuclear",19862326
...,...,...,...,...
42540,TGFBR2,INTERACTS_WITH,Transforming Growth Factor beta,26627606
42541,TGFBR2,INTERACTS_WITH,Transforming Growth Factor beta,17056535
42542,TGFBR2,INTERACTS_WITH,Transforming Growth Factor beta,10426193
42543,TGFBR2,INTERACTS_WITH,Transforming Growth Factor beta,26169354


In [39]:
#GENE_LIST = ['ATG10']

# Do not run full - too long

GENE_LIST = genes_test
query = """
        MATCH (g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt1:LiteratureTriple)-[]-(l2:LiteratureTerm)-[]-(lt2:LiteratureTriple)-[]-(l3:LiteratureTerm)
        WHERE g.name in {gene_list}
        WITH  g,l1,lt1, l2, lt2, l3
        MATCH (lt1)-[]-(lit:Literature) 
        RETURN l1.name as term1, lt1.predicate as pred1, l2.name as term2 ,  lt2.predicate as pred2, l3.name as term3 , lit.id

    """.format(gene_list=GENE_LIST)
#df= query_to_df(query)
#df

In [11]:
gene_list = genes_test 

# tophits in GWAS -> genes -> expressed in Breast tissue -> map to lit -> other term -> predicate to a disease
# where middle term contains 'breast'
query="""
MATCH (g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt1:LiteratureTriple)-[]-(l2:LiteratureTerm)-[]-(lt2:LiteratureTriple)-[]-(l3:LiteratureTerm)-[:MONDO_MAP_UMLS]-(d:Disease)
WHERE g.name in {} 
AND toLower(l2.name) contains 'breast'
WITH  g,l1,lt1, l2, lt2, l3, d
MATCH (lt1)-[]-(lit1:Literature) 
MATCH (lt2)-[]-(lit2:Literature) 
RETURN l1.name as term1, lt1.predicate as pred1,
        l2.name as term2 ,  lt2.predicate as pred2,
        l3.name as term3 , d.label as Disease,
        lit1.id, lit2.id
""".format(gene_list)

df= query_to_df(query)
df


MATCH (g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt1:LiteratureTriple)-[]-(l2:LiteratureTerm)-[]-(lt2:LiteratureTriple)-[]-(l3:LiteratureTerm)-[:MONDO_MAP_UMLS]-(d:Disease)
WHERE g.name in ['NEK10', 'ANKLE1', 'ABO', 'ATG10', 'ADCY3', 'CREB5', 'CHEK2', 'ZNF184', 'HSCB', 'HSPA4', 'KANSL1', 'ELL', 'TET2', 'CBX8', 'MCM8', 'SETBP1', 'SSBP4', 'MKL1', 'PHLDA3', 'EBF1', 'TLR1', 'MLLT10', 'TTC28', 'LGR6', 'PEX14', 'FTO', 'SUB1', 'KCNN4', 'RIN3', 'OR2A1-AS1', 'SF3B4', 'N4BP2L1', 'TRIM46', 'CCDC88C', 'ADCY9', 'LINC-PINT', 'SLC25A22', 'USHBP1', 'ADSSL1', 'TCF7L2', 'VGLL3', 'STXBP4', 'FAM46A', 'MTMR11', 'CDYL2', 'MDM4', 'ASTN2', 'MYEOV', 'ATXN7', 'RANBP9', 'ESR1', 'GRHL1', 'L3MBTL3', 'CDCA7L', 'PLA2G6', 'ULK3', 'ARRDC3', 'AMFR', 'FGFR2', 'CUX1', 'OTUD7B', 'BABAM1', 'KRTCAP2', 'FAM175A', 'NREP', 'CASP8', 'PIDD', 'ZMIZ1', 'SCAMP2', 'GIPR', 'ARHGEF5', 'ZBTB38', 'PIK3R3', 'PDE4D', 'CMSS1', 'RPL23AP53', 'MAGI3', 'LSP1', 'ATAD5', 'ZFPM2', 'SV2A', 'DNAJC1', 'RCCD1', 'TGFBR2', 'PRC1', 'NRIP1', 'TAC

,term1,pred1,term2,pred2,term3,Disease,lit1.id,lit2.id
0,ESR1,ASSOCIATED_WITH,"Breast Fibrocystic Change, Proliferative Type",COEXISTS_WITH,Breast Diseases,breast disease,16327431,17417639
1,ESR1,ASSOCIATED_WITH,"Breast Fibrocystic Change, Proliferative Type",AFFECTS,Breast Diseases,breast disease,16327431,11289127
2,ARHGEF5,AFFECTS,"Breast Fibrocystic Change, Proliferative Type",COEXISTS_WITH,Breast Diseases,breast disease,14662653,17417639
3,ARHGEF5,AFFECTS,"Breast Fibrocystic Change, Proliferative Type",AFFECTS,Breast Diseases,breast disease,14662653,11289127


In [17]:
# extarct sentences where those lit predicates come from:
def get_sentence_data(pmid):
    url = "https://melodi-presto.mrcieu.ac.uk/api/sentence/"
    params = { "pmid": pmid,}
    response = requests.post(url, data=json.dumps(params))
    res = response.json()
    df = pd.json_normalize(res)
    return df

sentence_data = get_sentence_data("17417639")
for ss in sentence_data["data"]:
    for s in ss:
        print(s["SUB_PRED_OBJ"])
        print("SENTENCE:", s["SENTENCE"])
        print("\n")

NameError: name 'requests' is not defined

In [18]:
sentence_data = get_sentence_data("11289127")
for ss in sentence_data["data"]:
    for s in ss:
        print(s["SUB_PRED_OBJ"])
        print("SENTENCE:", s["SENTENCE"])
        print("\n")
        

NameError: name 'requests' is not defined

In [ ]:
bc_all_gwas_to_path[bc_all_gwas_to_path['gene.name']=='ESR1']

In [ ]:
bc_er_pos_gwas_to_path[bc_er_pos_gwas_to_path['gene.name']=='ESR1']

In [ ]:
bc_er_neg_gwas_to_path[bc_er_neg_gwas_to_path['gene.name']=='ESR1']

In [ ]:
list(bc_er_pos_gwas_to_path[bc_er_pos_gwas_to_path['gene.name']=='ESR1']['Pathways'])

In [19]:
gene_list = genes_test 

# tophits in GWAS -> genes -> expressed in Breast tissue -> map to lit -> other term -> predicate to a disease
# where third term contains 'breast'
query="""
MATCH (g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt1:LiteratureTriple)-[]-(l2:LiteratureTerm)-[]-(lt2:LiteratureTriple)-[]-(l3:LiteratureTerm)-[:MONDO_MAP_UMLS]-(d:Disease)
WHERE g.name in {} 
AND toLower(l3.name) contains 'breast'
WITH  g,l1,lt1, l2, lt2, l3, d
MATCH (lt1)-[]-(lit1:Literature) 
MATCH (lt2)-[]-(lit2:Literature) 
RETURN l1.name as term1, lt1.predicate as pred1,
        l2.name as term2 ,  lt2.predicate as pred2,
        l3.name as term3 , d.label as Disease,
        lit1.id, lit2.id
""".format(gene_list)

df= query_to_df(query)
df
#df=df[df["term2"]!= "Genes"]
#df.sort_values(by=['Disease'])
#df[df["pred1"] not in ["COEXISTS_WITH"]]

NameError: name 'genes_test' is not defined

## Mapping enterobiasis drugs to true diseases via literature

In [7]:
enterobiasis_drugs_df = pd.read_csv("query_results/enterobiasis_drugs.tsv", sep = '\t')
enterobiasis_drugs = list(set(enterobiasis_drugs_df['drug.label']))

In [8]:
query = """
        MATCH (g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt:LiteratureTriple)-[]-(l2:LiteratureTerm)
        WHERE g.name in {gene_list}
        WITH  g,lt,l1,l2
        MATCH (lt)-[]-(lit:Literature) 
        RETURN l1.name as gene_as_litterm, lt.predicate as pred, l2.name as obj_as_litterm, lit.id ;
    """.format(gene_list=GENE_LIST)
df= query_to_df(query)
df

NameError: name 'GENE_LIST' is not defined

In [ ]:
MATCH (drug:Drug)-[OPENTARGETS_DRUG_TO_TARGET]->(g:Gene)<-[:TERM_TO_GENE]-(l1:LiteratureTerm)-[]-(lt1:LiteratureTriple)-[]-(l2:LiteratureTerm)-[]-(lt2:LiteratureTriple)-[]-(l3:LiteratureTerm)-[:MONDO_MAP_UMLS]-(d:Disease)
where drug.label = 'ABEMACICLIB'
//AND toLower(l2.name) contains 'breast'
WITH  drug, l1,lt1, l2, lt2, l3, d
where tolower(d.label) contains 'breast'
RETURN drug.label as DRUG, 
		l1.name as term1, lt1.predicate as pred1,
        l2.name as term2 ,  lt2.predicate as pred2,
        l3.name as term3